## 5.6 Carsy

Aplikacja wykorzystuje `BottomNavigation` oraz `DrawerNavigation`. Po raz kolejny wykorzystamy tylko statyczne dostarczone przez obiekt `dataProvider`. Wykorzystamy `RecyclerView` w kilku różnych konfiguracjach. Będzie to uboga wersja [Fuelio](https://play.google.com/store/apps/details?id=com.kajda.fuelio&hl=pl&gl=US)
- `RecyclerView` z grupowaniem elementów osiągniętym przez wykorzystanie dwóch `ViewHolder`

<table><tr><td><img src="https://media2.giphy.com/media/0lpf97nejk58XmgAP1/giphy.gif?cid=790b7611c2da1160570c8dd99550ec3cec763c3d4c38b025&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media2.giphy.com/media/hv5Ewg1qBwSj99Ju82/giphy.gif?cid=790b7611ccbb9fccb551bafa99c524b39c01a03d957a334b&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media4.giphy.com/media/FkSzT1Y6TNOtg89XKr/giphy.gif?cid=790b76119c5e65885f7be2c85549b516585eebeb4a8d4baa&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

### **BottomNavigation**

Rozpocznijmy od dodania trzech pustych fragmentów (`OverviewFragment`, `CalculatorsFragment`, `TimeLineFragment`) i nawigacji

`navigation.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/overviewFragment">

    <fragment
        android:id="@+id/calculatorsFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.CalculatorsFragment"
        android:label="fragment_calculators"
        tools:layout="@layout/fragment_calculators" />
    <fragment
        android:id="@+id/timeLineFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.TimeLineFragment"
        android:label="fragment_time_line"
        tools:layout="@layout/fragment_time_line" />
    <fragment
        android:id="@+id/overviewFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.OverviewFragment"
        android:label="fragment_overview"
        tools:layout="@layout/fragment_overview" />
</navigation>

Po raz kolejny dodajemy tylko fragmenty nie definiując dla nich akcji. `OverviewFragment` oznaczam jako fragment domowy. Musimy również utworzyć `menu` dla `BottomNavigation`

`bottom_menu.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/overviewFragment"
        android:title="@string/overview" />
    <item
        android:id="@id/timeLineFragment"
        android:title="@string/time_line" />

    <item
        android:id="@id/calculatorsFragment"
        android:title="@string/calculators" />
</menu>

Pamiętajmy, że `android:id` muszą być takie same jak w `navigation.xml` aby automatyczna nawigacja działała prawidłowo.

Przejdźmy do `MainActivity` i dodajmy nawigację oraz połączmy ją z `BottomNavigation`

In [ ]:
public class MainActivity extends AppCompatActivity {

    private NavController navController;

    private ActivityMainBinding binding;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        binding = ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());

        NavHostFragment navHostFragment = (NavHostFragment) getSupportFragmentManager()
                .findFragmentById(R.id.nav_host_fragment);

        if (navHostFragment != null) {
            navController = NavHostFragment.findNavController(navHostFragment);
        }

        NavigationUI.setupWithNavController(binding.bottomNavView, navController);
    }
}

W efekcie powinniśmy otrzymać podstawową nawigację z trzema ekranami.

<img src="https://media1.giphy.com/media/x0j1SX0qmejSUNRtHu/giphy.gif?cid=790b7611bcaddd3fe77b775f20dd38e5e0cfa788c1e83df7&rid=giphy.gif&ct=g" width="150" />

### **TimeLineFragment**

Chcemy przygotować listę w postaci osi czasu, która będzie zbliżona do tego co oferuje aplikacja **Fuelio**

<img src="https://play-lh.googleusercontent.com/Lm6SO1jsUF-VjsrfkIV9x-qpPIkIOIvgWkBGmxyZuHWPSbkYMS0oLhgoBk9wLXq6Xw=w5120-h2880" width="150" />

Zrobimy to w postaci `RecyclerView` z dwoma `ViewHolder`. Wpierw przygotujmy dane. Typ kosztu zdefiniujemy jako `enum class`, będziemy przechowywać w niej typ oraz ikonę.

In [ ]:
public enum CostType {
    REFUELING("Tankowanie", R.drawable.ic_fuel),
    SERVICE("Serwis", R.drawable.ic_car_repair),
    PARKING("Parking", R.drawable.ic_parking),
    INSURANCE("Ubezpieczenie", R.drawable.ic_general_cost),
    TICKET("Mandat", R.drawable.ic_ticket);

    private final String costType;
    private final int icon;

    CostType(String costType, int icon) {
        this.costType = costType;
        this.icon = icon;
    }

    public String getCostType() {
        return costType;
    }

    public int getIcon() {
        return icon;
    }
}

Ikony standardowo wybieramy spośród dostępnych w **Android Studio** (**New -> Vector Asset**). Model danych zawiera typ kosztu, datę oraz kwotę.

In [ ]:
public class Cost {
    private final CostType type;
    private final LocalDate date;
    private final int amount;

    public Cost(CostType type, LocalDate date, int amount){
        this.type = type;
        this.date = date;
        this.amount = amount;
    }

    public CostType getType() {
        return type;
    }

    public LocalDate getDate() {
        return date;
    }

    public int getAmount() {
        return amount;
    }
}

Ponieważ będziemy posiadać pojedynczy `RecyclerView` i dwa `ViewHolder`, musimy w jakiś sposób rozróżnić typ naszych danych - dla uproszczenia będziemy sortować po miesiącach (bez uwzględnienia lat). Więc chcemy zdefiniować dwa typy 
- `CostListItem` - klasa bazowa

In [ ]:
public abstract class CostListItem {
    public static final int TYPE_DATE = 0;
    public static final int TYPE_GENERAL = 1;

    abstract public int getType();
}

- `CostDateItem` - posiadający tylko datę po której będziemy sortować oraz na podstawie której utworzymy wersję layoutu dla `RecyclerView`

In [ ]:
public class CostDateItem extends CostListItem {
    private String date;

    public CostDateItem(String date){
        this.date = date;
    }

    public String getDate() {
        return date;
    }

    @Override
    public int getType() {
        return TYPE_DATE;
    }
}

- `CostGeneralItem` - każdy inny element dla którego również utworzymy layout

In [ ]:
public class CostGeneralItem extends CostListItem {
    private Cost cost;

    public Cost getCost() {
        return cost;
    }

    public CostGeneralItem(Cost cost){
        this.cost = cost;
    }


    @Override
    public int getType() {
        return TYPE_GENERAL;
    }
}

Przejdźmy do obiektu `DataProvider` - w pierwszym kroku utworzymy listę kosztów

In [ ]:
public final class DataProvider {
    private DataProvider() {
    }

    public static ArrayList<Cost> getGeneralCosts() {
        ArrayList<Cost> items = new ArrayList<>();
        for (int i = 0; i < 100; i++) {
            items.add(new Cost(
                    CostType.values()[new Random().nextInt(CostType.values().length)],
                    LocalDate.of(2022, new Random().nextInt(12) + 1, new Random().nextInt(28) + 1),
                    new Random().nextInt(5000)));
        }
        return items;
    }
}

Ponieważ mamy jeden `RecyclerView`, więc będziemy mieć jedną listę - musimy ją odpowiednio przygotować. Wykorzystamy `SortedMap` i posortujemy całość po miesiącach, następnie pogrupujemy względem miesięcy.

In [ ]:
private static final SortedMap<Month, ArrayList<Cost>> groupedHashMap = 
    groupDataIntoHashMap(getGeneralCosts());

private static SortedMap<Month, ArrayList<Cost>> groupDataIntoHashMap(ArrayList<Cost> itemList) {

    SortedMap<Month, ArrayList<Cost>> groupedHashMap = new TreeMap<>();
    for (Cost cost : itemList) {
        Month hashMapKey = cost.getDate().getMonth();
        if (groupedHashMap.containsKey(hashMapKey)) {
            Objects.requireNonNull(groupedHashMap.get(hashMapKey)).add(cost);
        } else {
            ArrayList<Cost> list = new ArrayList<>();
            list.add(cost);
            groupedHashMap.put(hashMapKey, list);
        }
    }
    return groupedHashMap;
}

Następnie napiszemy funkcję `getTimeLineList` zwracająca listę dla `RecyclerView`

In [ ]:
public static ArrayList<CostListItem> getTimeLineList() {

Utwórzmy listę pomocniczą

In [ ]:
ArrayList<CostListItem> items = new ArrayList<>();

Przechodzimy po wszystkich kluczach w naszej mapie

In [ ]:
for (Month date : groupedHashMap.keySet()) {

W pętli tworzę listę wszystkich elementów z danego miesiąca, następnie ją sortuję względem dnia miesiąca i każdy element `CostGeneralItem` dodaję do `list`

In [ ]:
ArrayList<Cost> groupingItems = groupedHashMap.get(date);
if (groupingItems != null) {
    groupingItems.sort(Comparator.comparingInt(o -> o.getDate().getDayOfMonth()));
    groupingItems.forEach(cost -> items.add(new CostGeneralItem(cost)));
}

Ostatnim elementem pętli będzie dodanie `CostDateItem`

In [ ]:
CostDateItem dateItem = new CostDateItem(date.name());
items.add(dateItem);

Na koniec wychodzimy z pętli `for` i zwracamy listę

In [ ]:
    }
    return items;
}

Pełna funkcja

In [ ]:
public static ArrayList<CostListItem> getTimeLineList() {
    ArrayList<CostListItem> items = new ArrayList<>();

    for (Month date : groupedHashMap.keySet()) {
        ArrayList<Cost> groupingItems = groupedHashMap.get(date);
        if (groupingItems != null) {
            groupingItems.sort(Comparator.comparingInt(o -> o.getDate().getDayOfMonth()));
            groupingItems.forEach(cost -> items.add(new CostGeneralItem(cost)));
        }
        CostDateItem dateItem = new CostDateItem(date.name());
        items.add(dateItem);
    }
    return items;
}

Podsumowując:
- tworzą listę do której wrzucę wszystkie elementy odpowiednio posortowane - lista zawiera elementy typu `CostListItem`
- przechodzę po kluczach wcześniej utworzonej mapy (miesiące)
- dodaję wszystkie elementy `CostGeneralItem` - wszystkie koszty dla danego miesiąca
- na koniec dodaję `CostDateItem` - miesiąc

Przejdźmy do `TimeLineAdapter` - klasa będzie przyjmowała w parametrze `Context` i rozszerzała `RecyclerView.Adapter`

In [ ]:
public class TimeLineAdapter extends RecyclerView.Adapter<RecyclerView.ViewHolder> {}

W pierwszej kolejności dodajmy listę elementów.

In [ ]:
private final ArrayList<CostListItem> itemList = DataProvider.getTimeLineList();

Zdefiniujmy dwie klasy `ViewHolder` - pierwszą będzie klasa dla elementu grupującego (tutaj jest to data - miesiąc)

In [ ]:
    static class DateViewHolder extends RecyclerView.ViewHolder{

        private final TextView dateTextView;

        public DateViewHolder(@NonNull View itemView) {
            super(itemView);

            dateTextView = itemView.findViewById(R.id.timeLineDateTextView);
        }

        public void bind(CostDateItem item){
            dateTextView.setText(item.getDate());
        }
    }

Dla tego `ViewHolder` definiujemy layout w pliku `date_item_timeline.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="16px"
        android:layout_height="0dp"
        android:layout_marginStart="24dp"
        android:background="?attr/colorPrimary"
        android:outlineAmbientShadowColor="?attr/colorPrimary"
        android:outlineSpotShadowColor="?attr/colorPrimary"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ImageView
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="19dp"
        android:layout_marginTop="24dp"
        android:background="@drawable/ic_round_circle"
        android:backgroundTint="?attr/colorPrimary"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/RVcolorBarView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="@+id/RVcolorBarView" />

    <TextView
        android:id="@+id/timeLineDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:layout_marginTop="24dp"
        android:fontFamily="serif"
        android:textColor="?attr/colorSecondary"
        android:padding="4dp"
        android:text="@string/date"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Jak widzimy mamy tylko jedno pole `TextView` wyświetlające nazwę miesiąca po którym grupujemy. Pozostałe dwa elementy są graficznym indykatorem - `View` jest linią o zadanej grubości i `ImageView` tutaj ma formę wypełnionego koła.

Zdefiniujmy również drugi `ViewHolder` dla pozostałych elementów na liście - tutaj chcemy pokazać typ kosztu, kwotę, pełną datę, oraz ikonę dla odpowiedniego typu kosztu.

In [ ]:
class GeneralViewHolder extends RecyclerView.ViewHolder{

    private final TextView costTypeTextView;
    private final TextView fullDateTextView;
    private final TextView amountTextView;
    private final ImageView iconImageView;

    public GeneralViewHolder(@NonNull View itemView) {
        super(itemView);

        costTypeTextView = itemView.findViewById(R.id.timeLineCostTypeNameTextView);
        fullDateTextView = itemView.findViewById(R.id.timeLineFullDateTextView);
        amountTextView = itemView.findViewById(R.id.timeLineCostAmountTextView);
        iconImageView = itemView.findViewById(R.id.iconTimeLineImageView);
    }

    public void bind(CostGeneralItem item){
        costTypeTextView.setText(item.getCost().getType().getCostType());
        fullDateTextView.setText(item.getCost().getDate().format(FormatterUtil.dateFormatter));
        amountTextView.setText(String.format("%s zł", item.getCost().getAmount()));
        iconImageView.setBackground(
            ContextCompat
            .getDrawable(context, item.getCost().getType().getIcon()));
    }
}

Będziemy również formatować nieco `String` przy dacie, więc zdefiniujmy `DateTimeFormatter`

In [ ]:
public final class FormatterUtil {
    private FormatterUtil(){}

    public static DateTimeFormatter dateFormatter = DateTimeFormatter.ofPattern("yyyy MMM dd");
}

Layout jest nieco bardziej skomplikowany

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="16px"
        android:layout_height="0dp"
        android:layout_marginStart="24dp"
        android:background="?attr/colorPrimary"
        android:outlineAmbientShadowColor="?attr/colorPrimary"
        android:outlineSpotShadowColor="?attr/colorPrimary"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ImageView
        android:id="@+id/imageView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="3.2dp"
        android:background="@drawable/ic_round_circle_big"
        android:backgroundTint="?attr/colorPrimary"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/RVcolorBarView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="@+id/RVcolorBarView" />

    <ImageView
        android:id="@+id/iconTimeLineImageView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="12dp"
        android:layout_marginTop="8dp"
        android:background="@drawable/ic_car_repair"
        android:backgroundTint="@color/white"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/imageView"
        app:layout_constraintStart_toStartOf="@+id/imageView"
        app:layout_constraintTop_toTopOf="@+id/imageView"
        app:layout_constraintVertical_bias="0.285" />

    <TextView
        android:id="@+id/timeLineCostTypeNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:padding="4dp"
        android:text="@string/costtype_name"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toEndOf="@+id/imageView"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/timeLineCostAmountTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="16dp"
        android:padding="4dp"
        android:text="1200 zł"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/timeLineFullDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:fontFamily="serif"
        android:paddingStart="4dp"
        android:text="@string/date"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toEndOf="@+id/imageView"
        app:layout_constraintTop_toBottomOf="@+id/timeLineCostTypeNameTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>

Zauważmy że znów mamy `View` o tych samych parametrch co w poprzednim layoucie co sprawia wrażenie ciągłości przez wszystkie elementy. Mamy również dwa `ImageView` - pierwszy zastosowałem jako okrągłe tło o jednolitym kolorze dla ikony obecnej w drugim.

Przejdźmy do metody `onCreateViewHolder` - zwrócimy `ViewHolder` dla zadanego `viewType`. Ponieważ będziemy wykorzystywać dwa, musimy zdefiniować jeszcze `viewType`.

In [ ]:
@NonNull
@Override
public RecyclerView.ViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {

    RecyclerView.ViewHolder viewHolder;
    LayoutInflater inflater = LayoutInflater.from(parent.getContext());

    switch (viewType){
        case CostListItem.TYPE_DATE:
            View dateView = inflater.inflate(R.layout.date_item_timeline, parent, false);
            viewHolder = new DateViewHolder(dateView);
            break;
        case CostListItem.TYPE_GENERAL:
            View generalView = inflater.inflate(R.layout.general_item_timeline, parent, false);
            viewHolder = new GeneralViewHolder(generalView);
            break;
        default:
            throw new IllegalStateException("Unexpected value: " + viewType);
    }

    return viewHolder;
}

Aby zdefiniować `viewType` musimy nadpisać metodę `getItemViewType`

In [ ]:
public int getItemViewType(int position) {
    return itemList.get(position).getType();
} // zwraca typ elementu na liście

Zaimplementujmy metodę `getItemCount`

In [ ]:
@Override
public int getItemCount() {
    return itemList.size();
}

Pozostaje implementacja metody `onBindViewHolder` - w zależności od typu `ViewHolder` chcemy przekazać `CostListItem` jako `CostGeneralItem` zawierający instancję `Cost` lub `CostDateItem` zawierający nazwę miesiąca jako `String`.

In [ ]:
override fun onBindViewHolder(holder: RecyclerView.ViewHolder, position: Int) {
    when (holder.itemViewType) {
        CostListItem.TYPE_DATE -> (holder as DateViewHolder).bind(
            item = itemList[position] as CostDateItem
        )
        CostListItem.TYPE_GENERAL -> (holder as GeneralViewHolder).bind(
            item = itemList[position] as CostGeneralItem
        )
    }
}

Przejdźmy do `TimeLineFragment` i dodajmy nasz RecyclerView

In [ ]:
@Override
public void onBindViewHolder(@NonNull RecyclerView.ViewHolder holder, int position) {
    switch (holder.getItemViewType()){
        case CostListItem.TYPE_DATE:
            CostDateItem costDateItem = (CostDateItem) itemList.get(position);
            DateViewHolder dateViewHolder = (DateViewHolder) holder;
            dateViewHolder.bind(costDateItem);
            break;
        case CostListItem.TYPE_GENERAL:
            CostGeneralItem costGeneralItem = (CostGeneralItem) itemList.get(position);
            GeneralViewHolder generalViewHolder = (GeneralViewHolder) holder;
            generalViewHolder.bind(costGeneralItem);
            break;
        default:
            throw new IllegalStateException("Unexpected type");
    }
}

Możemy przetestować aplikację

<img src="https://media4.giphy.com/media/S4iOJk9GSVcizP2tRw/giphy.gif?cid=790b76113b3cee84bee0e3484c2fbc03807cf26e022c376a&rid=giphy.gif&ct=g" width="150" />

### **Model danych**

Przygotujmy prosty model danych opisujący samochód

In [ ]:
public class Car {
    private final String name;
    private final String brand;
    private final String model;
    private final int yearOfProduction;
    private ArrayList<Cost> costs;

    public Car(String name, String brand, String model, int yearOfProduction, ArrayList<Cost> costs) {
        this.name = name;
        this.brand = brand;
        this.model = model;
        this.yearOfProduction = yearOfProduction;
        this.costs = costs;
    }

    public void setCosts(ArrayList<Cost> costs) {
        this.costs = costs;
    }

    public String getName() {
        return name;
    }

    public String getBrand() {
        return brand;
    }

    public String getModel() {
        return model;
    }

    public int getYearOfProduction() {
        return yearOfProduction;
    }

    public ArrayList<Cost> getCosts() {
        return costs;
    }
}


Poza standardowymi polami umieścimy również listę wszystkich kosztów (do której nie będziemy dodawać kolejnych wpisów). Musimy wprowadzić niewielkie zmiany w `DataProvider` i wystawić liste wszystkich samochodów. Zmienimy `generalCosts` na funkcję przyjmującą rozmiar listy.

In [ ]:
private static ArrayList<Cost> getGeneralCosts(int size) {
    ArrayList<Cost> items = new ArrayList<>();
    for (int i = 0; i < size; i++) {
        items.add(new Cost(
                CostType.values()[new Random().nextInt(CostType.values().length)],
                LocalDate.of(2022, new Random().nextInt(12) + 1, new Random().nextInt(28) + 1),
                new Random().nextInt(5000)));
    }
    return items;
}

Zmodyfikujemy również nieco funkcję `getTimeLineList`

In [ ]:
public static ArrayList<CostListItem> getTimeLineList(ArrayList<Cost> costs) {
    ArrayList<CostListItem> items = new ArrayList<>();

    final SortedMap<Month, ArrayList<Cost>> groupedHashMap = groupDataIntoHashMap(costs);

    for (Month date : groupedHashMap.keySet()) {
        ArrayList<Cost> groupingItems = groupedHashMap.get(date);
        if (groupingItems != null) {
            groupingItems.sort(Comparator.comparingInt(o -> o.getDate().getDayOfMonth()));
            groupingItems.forEach(cost -> items.add(new CostGeneralItem(cost)));
        }
        CostDateItem dateItem = new CostDateItem(date.name());
        items.add(dateItem);
    }
    return items;
}

private static SortedMap<Month, ArrayList<Cost>> groupDataIntoHashMap(ArrayList<Cost> itemList) {

    SortedMap<Month, ArrayList<Cost>> groupedHashMap = new TreeMap<>();
    for (Cost cost : itemList) {
        Month hashMapKey = cost.getDate().getMonth();
        if (groupedHashMap.containsKey(hashMapKey)) {
            Objects.requireNonNull(groupedHashMap.get(hashMapKey)).add(cost);
        } else {
            ArrayList<Cost> list = new ArrayList<>();
            list.add(cost);
            groupedHashMap.put(hashMapKey, list);
        }
    }
    return groupedHashMap;
}

Chcemy mieć możliwość pokazania listy wszystkich kosztów dla każdego samochodu. Zmieńmy sygnaturę klasy `TimeLineAdapter` - będzie przyjmować listę wszystkich kosztów `List<Cost>`

In [ ]:
public class TimeLineAdapter extends RecyclerView.Adapter<RecyclerView.ViewHolder> {

    private final Context context;
    private final ArrayList<CostListItem> itemList;

    public TimeLineAdapter(Context context, ArrayList<Cost> costs){
        this.context = context;
        this.itemList = DataProvider.getTimeLineList(costs);
        Collections.reverse(itemList);
    }
    ...
}

Ostatnim elementem będzie wystawienie listy wszystkich samochodów przez obiekt `DataProvider`

In [ ]:
public static ArrayList<Car> getCars(){
    ArrayList<Car> cars = new ArrayList<>();
    cars.add(new Car("Domowy", "Skoda", "Fabia", 2002, getGeneralCosts(100)));
    cars.add(new Car("Służbowy", "BMW", "Coupe", 2015, getGeneralCosts(50)));
    cars.add(new Car("Kolekcjonerski", "Fiat", "125p", 1985, getGeneralCosts(10)));
    return cars;
}

### **Menu rozwijane - Spinner**

Na `TimeLineFragment` będziemy zmieniać listę kosztów za pomocą `Spinner`. W pierwszym kroku dodajmy go do layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".fragments.TimeLineFragment">

    <Spinner
        android:id="@+id/cars_spinner"
        android:layout_width="match_parent"
        android:drawSelectorOnTop="true"
        android:background="@drawable/spinner_bg"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_marginTop="20dp"
        android:layout_marginStart="10dp"
        android:layout_marginEnd="10dp"
        android:layout_height="wrap_content" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/timeLineRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginTop="12dp" />

</LinearLayout>

W pliku `spinner_bg` w katalogu `drawable` definiujemy tło naszego spinnera

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<selector xmlns:android="http://schemas.android.com/apk/res/android"
    android:exitFadeDuration="@android:integer/config_mediumAnimTime">
    <item>
        <inset android:insetLeft="-1dp" android:insetRight="-1dp">
            <shape android:shape="rectangle">
                <stroke android:width="1dp"
                    android:color="@color/black" />
                <solid android:color="?attr/colorPrimary" />
                <corners android:radius="100dp"/>
            </shape>
        </inset>
    </item>
</selector>

Kolejnym krokiem będzie utworzenie layoutu pojedynczego elementu listy `Spinner` - będzie to pole `TextView`. W katalogu `layout` tworzymy nowy plik `spinner_layout`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<TextView xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/spinner_layout"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:ellipsize="marquee"
    android:fontFamily="sans-serif"
    android:gravity="center"
    android:textColor="?attr/colorOnPrimary"
    android:background="@drawable/spinner_bg"
    android:singleLine="true"
    android:text="CAR"
    android:padding="10dp"
    android:textSize="35sp" />

Pozostało utworzyć `Spinner` w metodzie `onViewVCreated` klasy `TimeLineFragment`. Będziemy przeładowywać cały adapter w `RecyclerView`, więc tworzymy odpowiednią zmienną

In [ ]:
public class TimeLineFragment extends Fragment {

    @Override
    public View onCreateView(LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_time_line, container, false);
    }

    @Override
    public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);
        RecyclerView recyclerView = view.findViewById(R.id.timeLineRecyclerView);
        recyclerView.setLayoutManager(new LinearLayoutManager(requireContext()));
        recyclerView.setAdapter(
            new TimeLineAdapter(
                    requireContext(), 
                    DataProvider.getCars().get(0).getCosts()));
    }
}

Przejdźmy do obsłużenia `Spinner`, tutaj również będziemy potrzebować odpowiedni adapter. Adapterem który wykorzystamy będzie `ArrayAdapter` przyjmuje on trzy argumenty
- `context`
- `layout`
- listę danych
Jako dane podajemy listę naszych nazw samochodów

In [ ]:
Spinner spinner = view.findViewById(R.id.cars_spinner);
ArrayAdapter<String> adapter = new ArrayAdapter<>(getContext(), R.layout.spinner_layout,
        DataProvider.getCars().stream().map(Car::getName).collect(Collectors.toList()));
spinner.setAdapter(adapter);

Następnie chcemy podmienić adapter i odświeżyć `RecyclerView` po wybraniu opcji na `Spinner` - w tym calu implementujemy `AdapterView.OnItemSelectedListener`. Ten interfejs zawiera dwie metody
- `onItemSelected`
- `onNothingSelected`

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);

    Spinner spinner = view.findViewById(R.id.cars_spinner);
    ArrayAdapter<String> adapter = new ArrayAdapter<>(getContext(), R.layout.spinner_layout,
            DataProvider.getCars().stream().map(Car::getName).collect(Collectors.toList()));
    spinner.setAdapter(adapter);
    spinner.setOnItemSelectedListener(new AdapterView.OnItemSelectedListener() {
        @Override
        public void onItemSelected(AdapterView<?> parent, View view, int position, long id) {}

        @Override
        public void onNothingSelected(AdapterView<?> parent) {

        }
    });
}

Nas interesuje tylko pierwsza, `onNothingSelected` pozostawimy pustą. Wywołamy metodę `swapAdapter` klasy `RecyclerView` - przyjmuje ona dwa parametry
- adapter - nowy adapter, który zastąpi poprzedni
- `removeAndRecycleExistingViews: Boolean` - jeśli ustawimy jako `true` wszystkie utworzone do tej pory obiekty `View` zostaną usunięte

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);
    RecyclerView recyclerView = view.findViewById(R.id.timeLineRecyclerView);
    recyclerView.setLayoutManager(new LinearLayoutManager(requireContext()));
    recyclerView.setAdapter(
        new TimeLineAdapter(requireContext(), 
                            DataProvider.getCars().get(0).getCosts()));

    Spinner spinner = view.findViewById(R.id.cars_spinner);
    ArrayAdapter<String> adapter = new ArrayAdapter<>(getContext(), R.layout.spinner_layout,
            DataProvider.getCars().stream().map(Car::getName).collect(Collectors.toList()));
    spinner.setAdapter(adapter);
    spinner.setOnItemSelectedListener(new AdapterView.OnItemSelectedListener() {
        @Override
        public void onItemSelected(AdapterView<?> parent, View view, int position, long id) {
            recyclerView.swapAdapter(
                new TimeLineAdapter(getContext(),
                                    DataProvider.getCars().get(position).getCosts()), 
                                    true);
        }

        @Override
        public void onNothingSelected(AdapterView<?> parent) {

        }
    });
}

Możemy przetestować aplikację

<img src="https://media2.giphy.com/media/Qo4yNLd6gn7090SWRl/giphy.gif?cid=790b7611251b7107f210b555df82ce9f979a76103dae2791&rid=giphy.gif&ct=g" width="150" />

### **CalculatorsFragment**

Dodamy możliwość skorzystania z trzech kalkulatorów pozwalających obliczyć podstawowe parametry na podstawie informacji o paliwie, spalaniu, cenie i dystansie. Kalkulatory będą współdzielić layout, zmianę będziemy dokonywać za pomocą `Spinner`.

Będziemy posługiwać się liczbami zmiennoprzecinkowymi, więc dodajmy zaokrąglenie do dwóch miejsc. W pliku `FormatterUtil` dodajemy

In [ ]:
public static DecimalFormat decimalFormat = new DecimalFormat("#.##");

Przygotujmy layout z formularzem oraz spinnerem

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:orientation="vertical"
    tools:context=".fragments.TimeLineFragment">

    <Spinner
        android:id="@+id/calculators_spinner"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="10dp"
        android:layout_marginTop="20dp"
        android:layout_marginEnd="10dp"
        android:background="@drawable/spinner_bg"
        android:drawSelectorOnTop="true" />

    <androidx.cardview.widget.CardView
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="10dp"
        android:layout_marginTop="20dp"
        android:layout_marginEnd="10dp"
        android:backgroundTint="?attr/colorPrimary"
        android:outlineProvider="background"
        app:cardCornerRadius="35dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:layout_marginTop="20dp"
            android:orientation="vertical">

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout1"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/odleg_o"
                app:boxStrokeColor="#fff"
                android:theme="@style/editText"
                app:hintTextColor="?attr/colorOnPrimary"
                android:textColorHint="#fff">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText1"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="?attr/colorOnPrimary"
                    android:textSize="18sp"
                    android:textColorHighlight="?attr/colorOnPrimary"
                    android:inputType="number" />
            </com.google.android.material.textfield.TextInputLayout>

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout2"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/cena_za_litr"
                app:boxStrokeColor="?attr/colorOnPrimary"
                android:theme="@style/editText"
                app:hintTextColor="?attr/colorOnPrimary"
                android:textColorHint="#fff">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText2"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="?attr/colorOnPrimary"
                    android:textSize="18sp"
                    android:textColorHighlight="@color/white"
                    android:inputType="numberDecimal" />
            </com.google.android.material.textfield.TextInputLayout>

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout3"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/spalanie"
                app:boxStrokeColor="?attr/colorOnPrimary"
                android:theme="@style/editText"
                app:hintTextColor="?attr/colorOnPrimary"
                android:textColorHint="?attr/colorOnPrimary">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText3"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="?attr/colorOnPrimary"
                    android:textSize="18sp"
                    android:textColorHighlight="?attr/colorOnPrimary"
                    android:inputType="numberDecimal" />
            </com.google.android.material.textfield.TextInputLayout>

            <LinearLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginTop="40dp"
                android:layout_marginStart="15dp"
                android:orientation="horizontal">

                <com.google.android.material.textview.MaterialTextView
                    android:id="@+id/textViewMainTitle"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textColor="?attr/colorOnPrimary"
                    android:text="MainTitle"
                    android:textStyle="bold"
                    android:textSize="24sp" />

                <com.google.android.material.textview.MaterialTextView
                    android:id="@+id/textViewMainValue"
                    android:layout_width="0dp"
                    android:layout_weight="1"
                    android:textAlignment="viewEnd"
                    android:layout_height="wrap_content"
                    android:textColor="?attr/colorOnPrimary"
                    android:textStyle="bold"
                    android:text=""
                    android:layout_marginEnd="20dp"
                    android:textSize="24sp" />

            </LinearLayout>
            <com.google.android.material.textview.MaterialTextView
                android:id="@+id/textViewBottomValue"
                android:layout_width="match_parent"
                android:textAlignment="viewEnd"
                android:layout_height="wrap_content"
                android:textColor="?attr/colorOnPrimary"
                android:text=""
                android:layout_marginEnd="20dp"
                android:layout_marginStart="20dp"
                android:textSize="18sp"
                android:layout_marginBottom="20dp"/>
        </LinearLayout>
    </androidx.cardview.widget.CardView>

    <com.google.android.material.button.MaterialButton
        android:id="@+id/calculateButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:backgroundTint="@android:color/holo_red_dark"
        android:layout_gravity="right"
        android:layout_marginEnd="35dp"
        android:text="Oblicz"/>

</LinearLayout>

Aby wykorzystać inną niż domyślna kolorystyka pól `EditText`, dodajmy do `themes.xml` własny styl

In [ ]:
<style name="editText" parent="Widget.MaterialComponents.TextInputLayout.OutlinedBox.Dense">
    <item name="boxStrokeColor">#fff</item>
    <item name="boxStrokeWidth">2dp</item>
</style>

Zwróćmy uwagę że pole `EditText` jest zastąpione przez wersję dodstępną w `Materials Design` - możemy zwrócić uwagę że w pliku `gradle` w bloku `dependencies` domyślnie mamy tą bibliotekę już dodaną

In [ ]:
implementation 'com.google.android.material:material:1.6.1'

Zwykłe pole `EditText` jest zastąpione przez dwa pole
- `TextInputLayout` - zawierający podpowiedź i kilka elementów graficznych
- `TextInputEditText` - sam `EditText`

Kolejnym elementem jest modyfikacja wejścia w tych polach
- `android:inputType="number"` - liczby całkowite - zostanie wyświetlona tylko klawiatura numeryczna
- `android:inputType="numberDecimal"` - liczby zmiennoprzecinkowe

Zamieszczonym przyciskiem będziemy wykonywać obliczenie.

Przejdźmy do klasy `CalculatorsFragment` i metody `onViewCreated` - dodajmy `Spinner`

In [ ]:
String[] titles = {"Koszt podróży", "Odległość", "Wymagane paliwo"};
Spinner spinner = view.findViewById(R.id.calculators_spinner);
ArrayAdapter<String> adapter = new ArrayAdapter<>(
    getContext(), 
    R.layout.spinner_layout, 
    titles);
spinner.setAdapter(adapter);

Wykorzystujemy ten sam layout co w poprzednim `Spinner`. Dodajmy implementację `onItemSelected`

In [ ]:
spinner.setOnItemSelectedListener(new AdapterView.OnItemSelectedListener() {
    @Override
    public void onItemSelected(AdapterView<?> parent, View v, int position, long id) {}

W pierwszym kroku ustawimy tekst pola `textViewMainTitle` jako nazwę kalkulatora oraz w zależności od wybranego kalkulatora zmienimy podpowiedź jednego pola `TextInputLayout`

In [ ]:
TextView textView = view.findViewById(R.id.textViewMainTitle);
textView.setText(titles[position]);

TextInputLayout textInputLayout = view.findViewById(R.id.editInputLayout1);
if (position == 1)
    textInputLayout.setHint("Paliwo [l]");
else
    textInputLayout.setHint("Odległość [km]");

Kolejnym krokiem będzie obsługa `onCLick` przycisku

In [ ]:
MaterialButton materialButton = view.findViewById(R.id.calculateButton);
materialButton.setOnClickListener(v1 -> {});

Rozpocznijmy od wyjęcia wszystkich danych z pól `EditText`

In [ ]:
EditText editText1 = view.findViewById(R.id.editText1);
EditText editText2 = view.findViewById(R.id.editText2);
EditText editText3 = view.findViewById(R.id.editText3);
Triple<String, String, String> input =new Triple<>(
    editText1.getText().toString(),
    editText2.getText().toString(),
    editText3.getText().toString()
);

Na podstawie tych danych (w zależności od wybranego kalkulatora) chcemy przeprowadzić odpowiednie obliczenia, zróbmy to w metodzie `calculators`

In [ ]:
private Pair<String, String> calculators (int position, Triple<String, String, String> values ){
    switch (position){
        case 0:
            String bottomString = FormatterUtil.decimalFormat.format(
                (Double.parseDouble(values.getThird()) / 100) * 
                Double.parseDouble(values.getFirst()));
            String mainString = FormatterUtil.decimalFormat.format(
                Double.parseDouble(bottomString) * 
                Double.parseDouble(values.getSecond()));
            return new Pair<>(mainString + " zł", bottomString + " l");
        case 1:
            bottomString = FormatterUtil.decimalFormat.format(
                Double.parseDouble(values.getFirst()) * 
                Double.parseDouble(values.getSecond()));
            mainString = FormatterUtil.decimalFormat.format(
                (100 * Double.parseDouble(values.getFirst())) / 
                Double.parseDouble(values.getThird()));
            return new Pair<>(mainString + " km", bottomString + " zł");
        case 2:
            mainString = FormatterUtil.decimalFormat.format(
                (Double.parseDouble(values.getFirst()) * 
                 Double.parseDouble(values.getThird())) / 100 );
            bottomString = FormatterUtil.decimalFormat.format(
                Double.parseDouble(mainString) * 
                Double.parseDouble(values.getSecond()));
            return new Pair<>(mainString + " l", bottomString + " zł");
        default:
            return new Pair<>("", "");
    }
}

Każdy kalkulator oblicza dwie wielkości, więc zwracam je jako `Pair<String, String>`. W meotdzie `onClick` sprawdzamy czy to co dastajemy z pól `EditText` nie jest puste i wywołujemy powyższą metodę - wyniki umieszczamy na odpowiednich polach `TextView`

In [ ]:
if (!input.getFirst().isEmpty() 
    && !input.getSecond().isEmpty() 
    && !input.getThird().isEmpty()){
    TextView mainTextView = view.findViewById(R.id.textViewMainValue);
    TextView bottomTextView = view.findViewById(R.id.textViewBottomValue);

    Pair<String, String> values = calculators(position, input);
    mainTextView.setText(values.first);
    bottomTextView.setText(values.second);
}

Pełna metoda `onViewCreated`:

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);

    String[] titles = {"Koszt podróży", "Odległość", "Wymagane paliwo"};
    Spinner spinner = view.findViewById(R.id.calculators_spinner);
    ArrayAdapter<String> adapter = new ArrayAdapter<>(
        getContext(), 
        R.layout.spinner_layout, 
        titles);
    adapter.setDropDownViewResource(R.layout.spinner_dropdown_layout);
    spinner.setAdapter(adapter);


    spinner.setOnItemSelectedListener(new AdapterView.OnItemSelectedListener() {
        @Override
        public void onItemSelected(AdapterView<?> parent, View v, int position, long id) {
            TextView textView = view.findViewById(R.id.textViewMainTitle);
            textView.setText(titles[position]);

            TextInputLayout textInputLayout = view.findViewById(R.id.editInputLayout1);
            if (position == 1)
                textInputLayout.setHint("Paliwo [l]");
            else
                textInputLayout.setHint("Odległość [km]");

            MaterialButton materialButton = view.findViewById(R.id.calculateButton);
            materialButton.setOnClickListener(v1 -> {
                EditText editText1 = view.findViewById(R.id.editText1);
                EditText editText2 = view.findViewById(R.id.editText2);
                EditText editText3 = view.findViewById(R.id.editText3);
                Triple<String, String, String> input =new Triple<>(
                    editText1.getText().toString(),
                    editText2.getText().toString(),
                    editText3.getText().toString()
                );

                if (!input.getFirst().isEmpty() 
                    && !input.getSecond().isEmpty() 
                    && !input.getThird().isEmpty()){
                    TextView mainTextView = view.findViewById(R.id.textViewMainValue);
                    TextView bottomTextView = view.findViewById(R.id.textViewBottomValue);

                    Pair<String, String> values = calculators(position, input);
                    mainTextView.setText(values.first);
                    bottomTextView.setText(values.second);
                }
            });
        }

        @Override
        public void onNothingSelected(AdapterView<?> parent) {

        }
    });
}

Możemy przetestować aplikację

<img src="https://media1.giphy.com/media/6Ee7Ygm11rzDTIbGYW/giphy.gif?cid=790b761123f95e9b3caef44cdf2de4af63e5e1962bbee76a&rid=giphy.gif&ct=g" width="150" />

Zmieńmy nieco wygląd aplikacji - usuńmy plik `themes.xml(night)`. W pliku `colors.xml` zdefiniujmy kilka kolorów

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<resources>
    <color name="purple_200">#FFBB86FC</color>
    <color name="purple_500">#FF6200EE</color>
    <color name="purple_700">#FF3700B3</color>
    <color name="teal_200">#FF03DAC5</color>
    <color name="teal_700">#FF018786</color>
    <color name="black">#FF000000</color>
    <color name="white">#FFFFFFFF</color>
    <color name="mtrl_textinput_default_box_stroke_color">#fff</color>
    <color name="green_500">#FF1EB980</color>
    <color name="dark_blue_900">#FF26282F</color>
    <color name="dark_blue_500">#FF004940</color>

</resources>

Następnie zmodyfikujmy plik `themes.xml`

In [ ]:
<resources xmlns:tools="http://schemas.android.com/tools">
    <!-- Base application theme. -->
    <style name="Theme.CarsyAppKotlin" parent="Theme.MaterialComponents.DayNight.NoActionBar">
        <!-- Primary brand color. -->
        <item name="colorPrimary">@color/dark_blue_500</item>
        <item name="colorPrimaryVariant">@color/dark_blue_900</item>
        <item name="colorOnPrimary">@color/teal_700</item>
        <!-- Secondary brand color. -->
        <item name="colorSecondary">@color/teal_700</item>
        <item name="colorSecondaryVariant">@color/teal_200</item>
        <item name="colorOnSecondary">@color/white</item>
        <!-- Status bar color. -->
        <item name="android:statusBarColor" tools:targetApi="l">?attr/colorPrimaryVariant</item>
        <!-- Customize your theme here. -->
    </style>

    <style name="editText" parent="Widget.MaterialComponents.TextInputLayout.OutlinedBox.Dense">
        <item name="boxStrokeColor">@color/teal_200</item>
        <item name="boxStrokeWidth">2dp</item>
    </style>
</resources>

Dodajmy osobny layout dla menu rozwijanego w `Spinner` (`spinner_dropdown_layoutx.xml`)

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<TextView xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/spinner_layout"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:ellipsize="marquee"
    android:fontFamily="sans-serif"
    android:gravity="center"
    android:textColor="@color/teal_200"
    android:background="@drawable/spinner_dropdown_bg"
    android:singleLine="true"
    android:text="CAR"
    android:padding="10dp"
    android:textSize="35sp" />

W klasach `TimeLineFragment` i `CalculatorsFragment` wykorzystajmy utworzony layout

In [ ]:
...
adapter.setDropDownViewResource(R.layout.spinner_dropdown_layout);
...

Przejdźmy do layoutu głównej aktywności i zmodyfikujmy kolorystykę `BottomNavigationView`

In [ ]:
<com.google.android.material.bottomnavigation.BottomNavigationView
    android:id="@+id/bottom_nav_view"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:background="@color/dark_blue_900"
    app:itemIconTint="@drawable/bottom_navigation_highlight"
    app:itemTextColor="@drawable/bottom_navigation_highlight"
    app:menu="@menu/bottom_menu" />

Pozbywamy się w ten sposób domyślnych ustawień dla podświetlenie zaznaczonej ikony oraz tekstu, więc zdefiniujmy własny w pliku `bottom_navigation_highlight.xml` (katalog `drawable`)

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<selector xmlns:android="http://schemas.android.com/apk/res/android">
    <item android:state_checked="true" android:color="@color/teal_200" />
    <item android:state_checked="false" android:color="@color/teal_700"/>
</selector>

W plikach `date_item_timeline.xml` i `general_item_timeline.xml` zmieniamy kolor tła na

In [ ]:
android:background="@color/dark_blue_900"

oraz kolor podpowiedzi na

In [ ]:
android:backgroundTint="@color/teal_200"

Modyfikujemy layout `CalculatorsFragment`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:orientation="vertical"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.TimeLineFragment">

    <Spinner
        android:id="@+id/calculators_spinner"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="10dp"
        android:popupBackground="@android:color/transparent"
        android:layout_marginTop="20dp"
        android:dropDownVerticalOffset="64dp"
        android:popupElevation="3dp"
        android:layout_marginEnd="10dp"
        android:background="@drawable/spinner_bg"
        android:drawSelectorOnTop="true" />

    <androidx.cardview.widget.CardView
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="10dp"
        android:layout_marginTop="20dp"
        android:layout_marginEnd="10dp"
        android:backgroundTint="?attr/colorPrimary"
        android:outlineProvider="background"
        app:cardCornerRadius="35dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:layout_marginTop="20dp"
            android:orientation="vertical">

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout1"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/odleg_o"
                app:boxStrokeColor="@color/teal_200"
                android:theme="@style/editText"
                app:hintTextColor="@color/teal_200"
                android:textColorHint="@color/teal_200">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText1"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="@color/teal_200"
                    android:textSize="18sp"
                    android:textColorHighlight="@color/teal_200"
                    android:inputType="number" />
            </com.google.android.material.textfield.TextInputLayout>

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout2"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/cena_za_litr"
                app:boxStrokeColor="@color/teal_200"
                android:theme="@style/editText"
                app:hintTextColor="@color/teal_200"
                android:textColorHint="@color/teal_200">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText2"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="@color/teal_200"
                    android:textSize="18sp"
                    android:textColorHighlight="@color/teal_200"
                    android:inputType="numberDecimal" />
            </com.google.android.material.textfield.TextInputLayout>

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout3"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/spalanie"
                app:boxStrokeColor="@color/teal_200"
                android:theme="@style/editText"
                app:hintTextColor="@color/teal_200"
                android:textColorHint="@color/teal_200">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText3"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="@color/teal_200"
                    android:textSize="18sp"
                    android:textColorHighlight="@color/teal_200"
                    android:inputType="numberDecimal" />
            </com.google.android.material.textfield.TextInputLayout>

            <LinearLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginTop="40dp"
                android:layout_marginStart="15dp"
                android:orientation="horizontal">

                <com.google.android.material.textview.MaterialTextView
                    android:id="@+id/textViewMainTitle"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textColor="@color/teal_200"
                    android:text="MainTitle"
                    android:textStyle="bold"
                    android:textSize="24sp" />

                <com.google.android.material.textview.MaterialTextView
                    android:id="@+id/textViewMainValue"
                    android:layout_width="0dp"
                    android:layout_weight="1"
                    android:textAlignment="viewEnd"
                    android:layout_height="wrap_content"
                    android:textColor="?attr/colorOnPrimary"
                    android:textStyle="bold"
                    android:text=""
                    android:layout_marginEnd="20dp"
                    android:textSize="24sp" />

            </LinearLayout>
            <com.google.android.material.textview.MaterialTextView
                android:id="@+id/textViewBottomValue"
                android:layout_width="match_parent"
                android:textAlignment="viewEnd"
                android:layout_height="wrap_content"
                android:textColor="@color/teal_200"
                android:text=""
                android:layout_marginEnd="20dp"
                android:layout_marginStart="20dp"
                android:textSize="18sp"
                android:layout_marginBottom="20dp"/>
        </LinearLayout>
    </androidx.cardview.widget.CardView>

    <com.google.android.material.button.MaterialButton
        android:id="@+id/calculateButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:backgroundTint="@android:color/holo_red_dark"
        android:textColor="@color/white"
        android:layout_gravity="right"
        android:textSize="18sp"
        android:layout_marginEnd="35dp"
        android:text="Oblicz"/>

</LinearLayout>

Również modyfikujemy layout `TimeLineFragment`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.TimeLineFragment">

    <Spinner
        android:id="@+id/cars_spinner"
        android:layout_width="match_parent"
        android:drawSelectorOnTop="true"
        android:popupBackground="@android:color/transparent"
        android:background="@drawable/spinner_bg"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_marginTop="20dp"
        android:popupElevation="3dp"
        android:layout_marginStart="10dp"
        android:dropDownVerticalOffset="64dp"
        android:layout_marginEnd="10dp"
        android:layout_height="wrap_content" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/timeLineRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginTop="12dp" />

</LinearLayout>

Po zmianach aplikacją powinna wyglądać następująco

<img src="https://media1.giphy.com/media/27zha8NAaWcBwWCJ6Q/giphy.gif?cid=790b7611b21116465fd1c582d6ad6ce899d6a4cb9e89cd4b&rid=giphy.gif&ct=g" width="150" />

### **OverviewFragment**

Do fragmentu zawierającego przegląd dodamy dwa `RecyclerView` zawierające podsumowanie wydatków oraz przegląd wszystkich samochodów. Jak również informację o całkowitych poniesionych kosztach. Rozpocznijmy od layoutu

Ponieważ nie wszystko może zostać zmieszczone na jednym ekranie naszym głównym elementem będzie `ScrollView`

In [ ]:
<ScrollView  xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:background="@color/dark_blue_900"
    android:layout_height="match_parent">

Pozostałe elementy umieścimy w `LinearLayout`

In [ ]:
<LinearLayout
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="vertical"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.OverviewFragment">

Dodajmy dwa `RecyclerView` z polami `TextView`

In [ ]:
    <TextView
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        android:text="@string/podsumowanie"
        android:gravity="center"
        android:textColor="@color/teal_200"
        android:textStyle="bold"
        android:layout_marginTop="24dp"
        />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/overviewCostsRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="12dp" />

    <TextView
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        android:text="@string/gara"
        android:gravity="center"
        android:textColor="@color/teal_200"
        android:textStyle="bold"
        android:layout_marginTop="24dp"
        />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/overviewCarsRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="12dp" />

Na końcu umieścimy `CardView` zawierający pola przeznaczone do przedstawienie całkowitych kosztów

In [ ]:

```xml
    <androidx.cardview.widget.CardView
        xmlns:app="http://schemas.android.com/apk/res-auto"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="12dp"
        android:layout_marginStart="40dp"
        android:layout_marginEnd="40dp"
        app:cardCornerRadius="45dp">

        <androidx.constraintlayout.widget.ConstraintLayout
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="4dp"
            android:layout_marginEnd="4dp"
            android:background="@color/dark_blue_500">

            <TextView
                android:id="@+id/overviewCarName"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:background="@color/green_500"
                android:gravity="center"
                android:padding="4dp"
                android:text="@string/koszty_ca_kowite"
                android:textColor="@color/black"
                android:textSize="24sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="parent" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/tankowanie"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewCarName" />

            <TextView
                android:id="@+id/overviewFueling"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewCarName" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/serwis"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewFueling" />

            <TextView
                android:id="@+id/overviewService"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewFueling" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/parkowanie"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewService" />

            <TextView
                android:id="@+id/overviewParking"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewService" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/ubezpieczenie"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewParking" />

            <TextView
                android:id="@+id/overviewInsurance"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewParking" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/mandat"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewInsurance" />

            <TextView
                android:id="@+id/overviewTicket"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewInsurance" />

        </androidx.constraintlayout.widget.ConstraintLayout>
    </androidx.cardview.widget.CardView>

</LinearLayout>

</ScrollView>

W następnym kroku przygotujmy layouty dla `RecyclerView`
- przedstawiający sumaryczne koszty ze względu na kategorią poniesione dla każdego posiadanego samochodu
- przedstawiający podstawowe informacje o posiadanych samochodach

`ite_recyclerview_cost_overview.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="40dp"
    android:layout_marginEnd="40dp"
    app:cardCornerRadius="45dp">

    <androidx.constraintlayout.widget.ConstraintLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="4dp"
        android:layout_marginEnd="4dp"
        android:background="@color/dark_blue_500">

        <TextView
            android:id="@+id/overviewRecyclerViewCarName"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:background="@color/green_500"
            android:gravity="center"
            android:padding="4dp"
            android:text="@string/car_name"
            android:textColor="@color/black"
            android:textSize="24sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/tankowanie"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewCarName" />

        <TextView
            android:id="@+id/overviewRecyclerViewFueling"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewCarName" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/serwis"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewFueling" />

        <TextView
            android:id="@+id/overviewRecyclerViewService"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewFueling" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/parkowanie"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewService" />

        <TextView
            android:id="@+id/overviewRecyclerViewParking"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewService" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/ubezpieczenie"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewParking" />

        <TextView
            android:id="@+id/overviewRecyclerViewInsurance"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewParking" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/mandat"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewInsurance" />

        <TextView
            android:id="@+id/overviewRecyclerViewTicket"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewInsurance" />

    </androidx.constraintlayout.widget.ConstraintLayout>
</androidx.cardview.widget.CardView>

`item_recyclerview_cars_overview.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="wrap_content"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp"
    app:cardCornerRadius="45dp">

    <androidx.constraintlayout.widget.ConstraintLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="4dp"
        android:layout_marginEnd="4dp"
        android:background="@color/dark_blue_500">

        <TextView
            android:id="@+id/overviewViewPagerCarName"
            android:layout_width="200dp"
            android:layout_height="wrap_content"
            android:background="@color/green_500"
            android:gravity="center"
            android:padding="4dp"
            android:text="@string/domowy"
            android:textColor="@color/black"
            android:textSize="24sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <TextView
            android:id="@+id/textView"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="24dp"
            android:padding="4dp"
            android:text="@string/marka"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerCarName" />

        <TextView
            android:id="@+id/overviewViewPagerBrand"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginEnd="24dp"
            android:gravity="end"
            android:padding="4dp"
            android:text="@string/skoda"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="@+id/overviewViewPagerCarName"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerCarName" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/model"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerBrand" />

        <TextView
            android:id="@+id/overviewViewPagerModel"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="@string/fabia"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="@+id/overviewViewPagerCarName"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerBrand" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/rok"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerModel" />

        <TextView
            android:id="@+id/overviewViewPagerYearOfProduction"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="@string/_1999"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="@+id/overviewViewPagerCarName"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerModel" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/suma_koszt_w"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerYearOfProduction" />

        <TextView
            android:id="@+id/overviewViewPagerTotalCosts"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="@+id/overviewViewPagerCarName"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerYearOfProduction" />

    </androidx.constraintlayout.widget.ConstraintLayout>
</androidx.cardview.widget.CardView>

Zmieńmy formatowanie wartości zmiennoprzecinkowych, przejdźmy do klasy `FormatterUtil`

In [ ]:
public static DecimalFormat decimalFormat = new DecimalFormat("###,###.## zł");

Pierwszym będzie `OverviewCarsAdapter`

In [ ]:
public class OverviewCarsAdapter extends RecyclerView.Adapter<OverviewCarsAdapter.ViewHolder> {

    private final ArrayList<Car> cars = DataProvider.getCars();

    public static class ViewHolder extends RecyclerView.ViewHolder {

        private final TextView carNameTextView;
        private final TextView brandTextView;
        private final TextView modelTextView;
        private final TextView yearTextView;
        private final TextView totalCostsTextView;

        public ViewHolder(@NonNull View itemView) {
            super(itemView);

            carNameTextView= itemView.findViewById(R.id.overviewViewPagerCarName);
            brandTextView = itemView.findViewById(R.id.overviewViewPagerBrand);
            modelTextView = itemView.findViewById(R.id.overviewViewPagerModel);
            yearTextView = itemView.findViewById(R.id.overviewViewPagerYearOfProduction);
            totalCostsTextView = itemView.findViewById(R.id.overviewViewPagerTotalCosts);
        }

        public void bind (Car item){
            carNameTextView.setText(item.getName());
            brandTextView.setText(item.getBrand());
            modelTextView.setText(item.getModel());
            yearTextView.setText(item.getYearOfProduction());
            totalCostsTextView.setText(FormatterUtil.decimalFormat.format(
                item.getCosts().stream()
                .map(Cost::getAmount)
                .reduce(0, Integer::sum))
            );
        }
    }

    @NonNull
    @Override
    public ViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new ViewHolder(LayoutInflater.from(parent.getContext()).inflate(
                R.layout.item_recyclerview_cars_overview, parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull ViewHolder holder, int position) {
        Car item = cars.get(position);
    }

    @Override
    public int getItemCount() {
        return cars.size();
    }
}


W funkcji `bind` pole `totalCostsTextView` będzie przechowywało informację o całkowitym poniesionym koszcie, aby to obliczyć na liście `costs` wywołujemy metodę `filter` - zwracającą listę wszystkich wartości (`getAmount`), `map` zwracającą listę wszystkich kosztów, oraz `reduce` zwracającą sumę.

Przejdźmy do `OverviewCostsAdapter`, chcemy obliczyć sumę kosztów każdego typu i przekazać wynik do `TextView`. Napiszmy metodę pomocniczą w klasie `ViewHolder` zwracającą sumę kosztów o zadanym typie.

In [ ]:
private String costValue(Car item, CostType costType){
    return FormatterUtil.decimalFormat.format(
            item.getCosts().stream()
                    .filter(i-> i.getType() == costType)
                    .map(Cost::getAmount)
                    .reduce(0, Integer::sum)
    );
}

Pełna klasa adaptera:

In [ ]:
public class OverviewCostsAdapter extends RecyclerView.Adapter<OverviewCostsAdapter.ViewHolder> {

    private final ArrayList<Car> cars = DataProvider.getCars();

    public static class ViewHolder extends RecyclerView.ViewHolder {

        private final TextView carNameTextView;
        private final TextView refuelingTextView;
        private final TextView serviceTextView;
        private final TextView parkingTextView;
        private final TextView insuranceTextView;
        private final TextView ticketTextView;

        public ViewHolder(@NonNull View itemView) {
            super(itemView);

            carNameTextView = itemView.findViewById(R.id.overviewRecyclerViewCarName);
            refuelingTextView = itemView.findViewById(R.id.overviewRecyclerViewFueling);
            serviceTextView = itemView.findViewById(R.id.overviewRecyclerViewService);
            parkingTextView = itemView.findViewById(R.id.overviewRecyclerViewParking);
            insuranceTextView = itemView.findViewById(R.id.overviewRecyclerViewInsurance);
            ticketTextView = itemView.findViewById(R.id.overviewRecyclerViewTicket);
        }

        public void bind (Car item){
            carNameTextView.setText(item.getName());
            refuelingTextView.setText(costValue(item, CostType.REFUELING));
            serviceTextView.setText(costValue(item, CostType.SERVICE));
            parkingTextView.setText(costValue(item, CostType.PARKING));
            insuranceTextView.setText(costValue(item, CostType.INSURANCE));
            ticketTextView.setText(costValue(item, CostType.TICKET));
        }

        private String costValue(Car item, CostType costType){
            return FormatterUtil.decimalFormat.format(
                    item.getCosts().stream()
                            .filter(i-> i.getType() == costType)
                            .map(Cost::getAmount)
                            .reduce(0, Integer::sum)
            );
        }
    }

    @NonNull
    @Override
    public ViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new ViewHolder(LayoutInflater.from(parent.getContext()).inflate(
                R.layout.item_recyclerview_cost_overview, parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull ViewHolder holder, int position) {
        Car item = cars.get(position);
        holder.bind(item);
    }

    @Override
    public int getItemCount() {
        return cars.size();
    }
}


Do metody `onViewCreated` klasy `OverviewFragment` dodajemy `RecyclerView`

In [ ]:
    public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);

        RecyclerView costsRecyclerView = view.findViewById(R.id.overviewCostsRecyclerView);
        costsRecyclerView.setLayoutManager(
            new LinearLayoutManager(getContext(), LinearLayoutManager.HORIZONTAL, false));
        costsRecyclerView.setAdapter(new OverviewCostsAdapter());

        RecyclerView carsRecyclerView = view.findViewById(R.id.overviewCarsRecyclerView);
        carsRecyclerView.setLayoutManager(
            new LinearLayoutManager(getContext(), LinearLayoutManager.HORIZONTAL, false));
        carsRecyclerView.setAdapter(new OverviewCarsAdapter());
    }

Następnie chcemy dostać całkowitą sumę kosztów dla każdego typu (dla wszystkich samochodów), napiszmy metodę pomocniczą

In [ ]:
private int totalValue(CostType costType){
    return  DataProvider.getCars().stream()
            .flatMap(list -> list.getCosts().stream())
            .filter(i-> i.getType() == costType)
            .map(Cost::getAmount)
            .reduce(0, Integer::sum);
}

Ponieważ każdy obierkt `Car` zawiera `List<Cost>`, wykonując `map` dostaniemy `List<List<Cost>>`. Będziemy sumować wszystkie koszty o zadanym typie, więc chcemy mieć jedną listę - w tym celu wykorzystujemy `flatMap`, która zwróci `List<Cost>`. Dalsze krokie są takie same jak poprzednio.

In [ ]:
public class OverviewFragment extends Fragment {

    @Override
    public View onCreateView(LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_overview, container, false);
    }

    @Override
    public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);

        RecyclerView costsRecyclerView = view.findViewById(R.id.overviewCostsRecyclerView);
        costsRecyclerView.setLayoutManager(
            new LinearLayoutManager(getContext(), LinearLayoutManager.HORIZONTAL, false));
        costsRecyclerView.setAdapter(new OverviewCostsAdapter());

        RecyclerView carsRecyclerView = view.findViewById(R.id.overviewCarsRecyclerView);
        carsRecyclerView.setLayoutManager(
            new LinearLayoutManager(getContext(), LinearLayoutManager.HORIZONTAL, false));
        carsRecyclerView.setAdapter(new OverviewCarsAdapter());

        TextView refuelingTextView = view.findViewById(R.id.overviewFueling);
        refuelingTextView.setText(
            FormatterUtil.decimalFormat.format(totalValue(CostType.REFUELING)));

        TextView serviceTextView = view.findViewById(R.id.overviewService);
        serviceTextView.setText(
            FormatterUtil.decimalFormat.format(totalValue(CostType.SERVICE)));

        TextView parkingTextView = view.findViewById(R.id.overviewParking);
        parkingTextView.setText(
            FormatterUtil.decimalFormat.format(totalValue(CostType.PARKING)));

        TextView insuranceTextView = view.findViewById(R.id.overviewInsurance);
        insuranceTextView.setText(
            FormatterUtil.decimalFormat.format(totalValue(CostType.INSURANCE)));

        TextView ticketTextView = view.findViewById(R.id.overviewTicket);
        ticketTextView.setText(
            FormatterUtil.decimalFormat.format(totalValue(CostType.TICKET)));

    }

    private int totalValue(CostType costType){
        return  DataProvider.getCars().stream()
                .flatMap(list -> list.getCosts().stream())
                .filter(i-> i.getType() == costType)
                .map(Cost::getAmount)
                .reduce(0, Integer::sum);
    }
}

Możemy przetestować aplikację

<table><tr><td><img src="https://media2.giphy.com/media/0lpf97nejk58XmgAP1/giphy.gif?cid=790b7611c2da1160570c8dd99550ec3cec763c3d4c38b025&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/hv5Ewg1qBwSj99Ju82/giphy.gif?cid=790b7611ccbb9fccb551bafa99c524b39c01a03d957a334b&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media4.giphy.com/media/FkSzT1Y6TNOtg89XKr/giphy.gif?cid=790b76119c5e65885f7be2c85549b516585eebeb4a8d4baa&rid=giphy.gif&ct=g" width="150" /></td></tr></table>